<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#[fileinput]通过重定向/管道/文件接受输入" data-toc-modified-id="[fileinput]通过重定向/管道/文件接受输入-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>[fileinput]通过重定向/管道/文件接受输入</a></span></li><li><span><a href="#[raise-SystemExit('It-failed!')]终止程序并给出错误信息" data-toc-modified-id="[raise-SystemExit('It-failed!')]终止程序并给出错误信息-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>[raise SystemExit('It failed!')]终止程序并给出错误信息</a></span></li><li><span><a href="#[argparse]解析命令行选项" data-toc-modified-id="[argparse]解析命令行选项-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>[argparse]解析命令行选项</a></span></li><li><span><a href="#[getpass]运行时弹出密码输入提示" data-toc-modified-id="[getpass]运行时弹出密码输入提示-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>[getpass]运行时弹出密码输入提示</a></span></li><li><span><a href="#[os.get_terminal_size()]获取终端的大小" data-toc-modified-id="[os.get_terminal_size()]获取终端的大小-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>[os.get_terminal_size()]获取终端的大小</a></span></li><li><span><a href="#[subprocess]{-windows-底下-bash-的命令沒有反應}执行外部命令并获取它的输出" data-toc-modified-id="[subprocess]{-windows-底下-bash-的命令沒有反應}执行外部命令并获取它的输出-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>[subprocess]{ windows 底下 bash 的命令沒有反應}执行外部命令并获取它的输出</a></span><ul class="toc-item"><li><span><a href="#補充" data-toc-modified-id="補充-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>補充</a></span></li></ul></li><li><span><a href="#[shutil]复制或者移动文件和目录" data-toc-modified-id="[shutil]复制或者移动文件和目录-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>[shutil]复制或者移动文件和目录</a></span><ul class="toc-item"><li><span><a href="#讨论" data-toc-modified-id="讨论-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>讨论</a></span></li><li><span><a href="#補充" data-toc-modified-id="補充-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>補充</a></span></li></ul></li><li><span><a href="#[shutil-.make_archive()-;-.unpack_archive]创建和解压归档文件" data-toc-modified-id="[shutil-.make_archive()-;-.unpack_archive]创建和解压归档文件-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>[shutil .make_archive() ; .unpack_archive]创建和解压归档文件</a></span></li><li><span><a href="#[os.walk()]通过文件名查找文件" data-toc-modified-id="[os.walk()]通过文件名查找文件-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>[os.walk()]通过文件名查找文件</a></span></li><li><span><a href="#[configparser.ConfigParser()-;-.ini]读取配置文件" data-toc-modified-id="[configparser.ConfigParser()-;-.ini]读取配置文件-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>[configparser.ConfigParser() ; .ini]读取配置文件</a></span><ul class="toc-item"><li><span><a href="#[ini]讨论" data-toc-modified-id="[ini]讨论-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>[ini]讨论</a></span></li></ul></li><li><span><a href="#[logging]给简单脚本增加日志功能" data-toc-modified-id="[logging]给简单脚本增加日志功能-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>[logging]给简单脚本增加日志功能</a></span></li><li><span><a href="#[logging]给函数库增加日志功能" data-toc-modified-id="[logging]给函数库增加日志功能-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>[logging]给函数库增加日志功能</a></span><ul class="toc-item"><li><span><a href="#讨论" data-toc-modified-id="讨论-12.1"><span class="toc-item-num">12.1&nbsp;&nbsp;</span>讨论</a></span></li></ul></li><li><span><a href="#[Timer]实现一个计时器" data-toc-modified-id="[Timer]实现一个计时器-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>[Timer]实现一个计时器</a></span><ul class="toc-item"><li><span><a href="#讨论" data-toc-modified-id="讨论-13.1"><span class="toc-item-num">13.1&nbsp;&nbsp;</span>讨论</a></span></li></ul></li><li><span><a href="#限制内存和CPU的使用量" data-toc-modified-id="限制内存和CPU的使用量-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>限制内存和CPU的使用量</a></span></li><li><span><a href="#[webbrowser]启动一个WEB浏览器" data-toc-modified-id="[webbrowser]启动一个WEB浏览器-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>[webbrowser]启动一个WEB浏览器</a></span></li></ul></div>

# [fileinput]通过重定向/管道/文件接受输入

在命令行中传递一个文件名给该脚本

In [8]:
%%file filein.py

import fileinput
import subprocess

# ===== sys ======
# python "...\filein.py" FileinputTest.txt
import sys
a=sys.argv[0]
b=sys.argv[1]

print("filename:",a)
print("param1:",b)

with fileinput.input(b) as f_input:
    for line in f_input:
        print(line, end='')

# ===== input =====
#file = input("請輸入...")
#print(file)
#
#with fileinput.input(file) as f_input:
#    for line in f_input:
#        print(line, end='')
        
# 暫停        
subprocess.call("pause",shell=True)

Overwriting filein.py


In [9]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))

# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'filein.py')
cmd = "python " + f'"{DIR}"'  + ' FileinputTest.txt' # sys.argv[1]

# 印出 cmd DIR
print(cmd,BASE_DIR,sep='\n')

#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

# 暫停(需在要執行的 py 上)
#subprocess.call("pause",shell=True)
#os.system("pause")

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理\filein.py" FileinputTest.txt
D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理


`fileinput.input()` 创建并返回一个 `FileInput` 类的实例。 

该实例除了拥有一些有用的帮助方法外，它还可被当做一个上下文管理器使用。 

通过将它作为一个上下文管理器使用，可以确保它不再使用时文件能自动关闭。

因此，整合起来，如果我们要写一个打印多个文件输出的脚本，那么我们需要在输出中包含文件名和行号

In [34]:
import fileinput

with fileinput.input('FileinputTest.txt') as f:
    for line in f:
        print(f.filename(), f.lineno(), line, end='') 

FileinputTest.txt 1 1: AAAAA
FileinputTest.txt 2 2: BBBBB
FileinputTest.txt 3 3: CCCCC
FileinputTest.txt 4 4: DDDDD


# [raise SystemExit('It failed!')]终止程序并给出错误信息

抛出一个 `SystemExit` 异常，使用错误消息作为参数。

例如：

```py
raise SystemExit('It failed!')
```

它会将消息在 `sys.stderr` 中打印，然后程序以状态码1退出

当你想要终止某个程序时，你可能会像下面这样写：

```py
import sys
sys.stderr.write('It failed!\n')
raise SystemExit(1)
```

如果你直接将消息作为参数传给 `SystemExit()` ，那么你可以省略其他步骤， 

比如 `import` 语句或将错误消息写入 `sys.stderr`

# [argparse]解析命令行选项

首先要创建一个 `ArgumentParser` 实例， 并使用 `add_argument()` 方法声明你想要支持的选项。 

在每个 `add_argument()` 调用中， `dest` 参数指定解析结果被指派给属性的名字。

`metavar` 参数被用来生成帮助信息。

`action` 参数指定跟属性对应的处理逻辑， 通常的值为 `store` ,被用来存储某个值或将多个参数值收集到一个列表中。 

---

`nargs='*'` 下面的参数收集所有剩余的命令行参数到一个列表中。

在本例中它被用来构造一个文件名列表：

```py
parser.add_argument(dest='filenames',metavar='filename', nargs='*')
```

---

`action='store_true'` 下面的参数根据参数是否存在来设置一个 `Boolean` 标志：

```py
parser.add_argument('-v', dest='verbose', action='store_true',
                    help='verbose mode')
```

---

`action='store'` 下面的参数接受一个单独值并将其存储为一个字符串：

```py
parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')
```

---

`action='append'` 下面的参数说明允许某个参数重复出现多次，并将它们追加到一个列表中去。 

`required` 标志表示该参数至少要有一个。

`-p` 和 `--pat` 表示两个参数名形式都可使用。

```py
parser.add_argument('-p', '--pat',metavar='pattern', required=True,
                    dest='patterns', action='append',
                    help='text pattern to search for')
```

---

最后， `choices={'slow','fast'}` 下面的参数说明接受一个值，但是会将其和可能的选择值做比较，以检测其合法性：

```py
parser.add_argument('--speed', dest='speed', action='store',
                    choices={'slow','fast'}, default='slow',
                    help='search speed')
```

---

一旦参数选项被指定，你就可以执行 `parser.parse()` 方法了。 它会处理 `sys.argv` 的值并返回一个结果实例。 

每个参数值会被设置成该实例中` add_argument()` 方法的 `dest` 参数指定的属性值。


In [14]:
%%file search.py
'''
Hypothetical command-line tool for searching a collection of
files for one or more text patterns.
'''

import os

import argparse
parser = argparse.ArgumentParser(description='Search some files')

parser.add_argument(dest='filenames',metavar='filename', nargs='*')

# required=True 必填
parser.add_argument('-p', '--pat',metavar='pattern', required=True,
                    dest='patterns', action='append',
                    help='text pattern to search for')

parser.add_argument('-v', dest='verbose', action='store_true',
                    help='verbose mode')

parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')

parser.add_argument('--speed', dest='speed', action='store',
                    choices={'slow','fast'}, default='slow',
                    help='search speed')

args = parser.parse_args()

# Output the collected arguments
print(args.filenames)
print(args.patterns)
print(args.verbose)
print(args.outfile)
print(args.speed)

os.system("pause")

Overwriting search.py


In [24]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))

# =============== cmd 參數 ================
# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'search.py')
cmd = "python " + f'"{DIR}"' + ' -v -p spam -p eggs foo.txt'
# ' -v -p spam --pat=eggs foo.txt bar.txt -o results --speed=fast'

# 印出 cmd DIR
print(cmd,BASE_DIR,sep='\n')

#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

# 暫停(需在要執行的 py 上)
#subprocess.call("pause",shell=True)
#os.system("pause")

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理\search.py" -v -p spam -p eggs foo.txt
D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理


```py
ArgumentParser(
    prog=None, # program 的名字，None 就檔名
    usage=None, # None 就根據你設定的參數產生相對應的說明
    description=None, # 描述
    epilog=None # 補充說明 ex: epilog="see the doc: https://..."
    )
```

位置參數 (positional argument)

    parser.add_argument(dest='filenames',metavar='filename', nargs='*')
    
    # nargs='*' 任意位置(無前綴)的參數
     foo.txt

選擇性參數 (optional argument) 根據前綴來指定

    parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')
                    
    -o results

# [getpass]运行时弹出密码输入提示

`Python` 的 `getpass` 模块正是你所需要的。

你可以让你很轻松的弹出密码输入提示， 并且不会在用户终端回显密码。

下面是具体代码

In [35]:
import getpass

user = getpass.getuser() # 'po390' # C:\Users\po390>
passwd = getpass.getpass()

def svc_login( passwd):
    if  passwd == "123":
        return True
    else:
        return False

if svc_login( passwd):    
    print('Yay!')
else:
    print('Boo!')
    
print(user)

········
Yay!
po390


如果你想显示的弹出用户名输入提示，使用内置的 `input` 函数：

```py
user = input('Enter your username: ')
```

# [os.get_terminal_size()]获取终端的大小

使用 `os.get_terminal_size()` 函数来做到这一点。

In [37]:
import os
sz = os.get_terminal_size()

sz
sz.columns
sz.lines

os.terminal_size(columns=120, lines=30)

120

30

# [subprocess]{ windows 底下 bash 的命令沒有反應}执行外部命令并获取它的输出

和 `shell` 有關 ， 在 `windows` 底下 `bash` 的命令沒有反應，

且預設編碼非 `utf8` 輸出也有困難

## 補充

In [18]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))

# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'filein.py')
cmd = "python " + f'"{DIR}"'  + ' FileinputTest.txt' # sys.argv[1]

# 印出 cmd DIR
print(cmd,BASE_DIR,sep='\n')

#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subp = subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

# 暫停(需在要執行的 py 上)
#subprocess.call("pause",shell=True)
#os.system("pause")

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理\filein.py" FileinputTest.txt
D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理



**參考** [Python3 subprocess](https://www.runoob.com/w3cnote/python3-subprocess.html)

In [20]:
import subprocess
def runcmd(command):
    ret = subprocess.run(command,shell=True,stdout=subprocess.PIPE,stderr=subprocess.PIPE,encoding="utf-8",timeout=1)
    if ret.returncode == 0:
        print("success:",ret)
    else:
        print("error:",ret)


runcmd(["dir","/b"])#序列参数

Exception in thread Thread-16:
Traceback (most recent call last):
  File "D:\anaconda\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "D:\anaconda\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "D:\anaconda\lib\subprocess.py", line 1063, in _readerthread
    buffer.append(fh.read())
  File "D:\anaconda\lib\codecs.py", line 321, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa5 in position 62: invalid start byte



IndexError: list index out of range

In [23]:
import time
import subprocess

def cmdfun(command):
    subp = subprocess.Popen(command,shell=True,stdout=subprocess.PIPE,stderr=subprocess.PIPE,encoding="utf-8")
    subp.wait(2)
    # 检查进程是否终止
    if subp.poll() == 0:
        # communicate(input,timeout) 和子进程交互，发送和读取数据
        print(subp.communicate()[1])
    else:
        print("失败")

cmdfun("python --version")


Python 3.6.5 :: Anaconda custom (64-bit)



In [24]:
cmd
cmdfun(cmd)

'python "D:\\CODE\\GitHub\\py\\資料結構\\py3-cookbook\\脚本编程与系统管理\\filein.py" FileinputTest.txt'

失败


# [shutil]复制或者移动文件和目录

你想要复制或移动文件和目录，但是又不想调用 `shell` 命令。

`shutil` 模块有很多便捷的函数可以复制文件和目录。使用起来非常简单

```py
import shutil

# Copy src to dst. (cp src dst)
shutil.copy(src, dst)

# 復制文件的內容以及文件的所有狀態信息。先copyfile後copystat
# Copy files, but preserve metadata (cp -p src dst)
shutil.copy2(src, dst)

# Copy directory tree (cp -R src dst)
shutil.copytree(src, dst)

# Move src to dst (mv src dst)
shutil.move(src, dst)
```

这些函数的参数都是字符串形式的文件或目录名。 底层语义模拟了类似的 `Unix` 命令

如果你只想复制符号链接本身，那么需要指定关键字参数 `follow_symlinks` ,如下：

如果你想保留被复制目录中的符号链接，像这样做：

```py
shutil.copytree(src, dst, symlinks=True)
```

`copytree()` 可以让你在复制过程中选择性的忽略某些文件或目录。 

你可以提供一个忽略函数，接受一个目录名和文件名列表作为输入，返回一个忽略的名称列表。

例如：

```py
def ignore_pyc_files(dirname, filenames):
    return [name in filenames if name.endswith('.pyc')]

shutil.copytree(src, dst, ignore=ignore_pyc_files)
```

由于忽略某种模式的文件名是很常见的，因此一个便捷的函数 `ignore_patterns()` 已经包含在里面了。

例如：

```py
shutil.copytree(src, dst, ignore=shutil.ignore_patterns('*~', '*.pyc'))
```

## 讨论

你通常不会去使用 `shutil.copytree()` 函数来执行系统备份。 

当处理文件名的时候，最好使用 `os.path` 中的函数来确保最大的可移植性（特别是同时要适用于 `Unix` 和 `Windows` ）

```py
>>> filename = '/Users/guido/programs/spam.py'
>>> import os.path
>>> os.path.basename(filename)
'spam.py'
>>> os.path.dirname(filename)
'/Users/guido/programs'
>>> os.path.split(filename)
('/Users/guido/programs', 'spam.py')
>>> os.path.join('/new/dir', os.path.basename(filename))
'/new/dir/spam.py'
>>> os.path.expanduser('~/guido/programs/spam.py')
'/Users/guido/programs/spam.py'
>>>
```

使用 `copytree()` 复制文件夹的一个棘手的问题是对于错误的处理。

例如，在复制过程中，函数可能会碰到损坏的符号链接，因为权限无法访问文件的问题等等。

为了解决这个问题，所有碰到的问题会被收集到一个列表中并打包为一个单独的异常，到了最后再抛出。 

下面是一个例子：

```py
try:
    shutil.copytree(src, dst)
except shutil.Error as e:
    for src, dst, msg in e.args[0]:
         # src is source name
         # dst is destination name
         # msg is error message from exception
         print(dst, src, msg)
```         
         
如果你提供关键字参数 `ignore_dangling_symlinks=True` ， 这时候 `copytree()` 会忽略掉无效符号链接。



## 補充

**Python中shutil模塊的常用文件操作函數用法示例**

https://www.itread01.com/articles/1476166838.html
    

`os` 模塊提供了對目錄或者文件的 新建/刪除/查看 文件屬性

```py
shutil.copyfileobj(fsrc, fdst[, length=16*1024])
# copy文件內容到另一個文件，可以copy指定大小的內容
```  

其中 `fsrc` ， `fdst` 都是文件對象，都需要打開後才能進行復制操作

```py
import shutil
f1=open('name','r')
f2=open('name_copy','w+')
shutil.copyfileobj(f1,f2,length=16*1024)
```

```py
# copyfile調用了copyfileobj
shutil.copyfile(src,dst)

# 僅copy權限，不更改文件內容，組和用戶
shutil.copymode(src,dst)  

# 復制所有的狀態信息，包括權限，組，用戶，時間等
shutil.copystat(src,dst)   

# 復制文件的內容以及權限，先copyfile後copymode
shutil.copy(src,dst) 

# 復制文件的內容以及文件的所有狀態信息。先copyfile後copystat
shutil.copy2(src,dst)

# 遞歸的復制文件內容及狀態信息
shutil.copytree(
    src, 
    dst, 
    symlinks=False, 
    ignore=None, 
    copy_function=copy2,
    ignore_dangling_symlinks=False
)  

# 遞歸地刪除文件
shutil.rmtree(path, ignore_errors=False, onerror=None) 

# 遞歸的移動文件
shutil.move(src, dst)   

make_archive(
    base_name,           # 壓縮打包後的文件名或者路徑名
    format,              # 壓縮或者打包格式    "zip", "tar", "bztar"or "gztar"
    root_dir=None,       # 將哪個目錄或者文件打包（也就是源文件）
    base_dir=None, 
    verbose=0,dry_run=0, 
    owner=None, 
    group=None, 
    logger=None
) 
```

# [shutil .make_archive() ; .unpack_archive]创建和解压归档文件

`shutil` 模块拥有两个函数—— `make_archive()` 和 `unpack_archive()` 可派上用场。 例如：

```py
>>> import shutil
>>> shutil.unpack_archive('Python-3.3.0.tgz')

>>> shutil.make_archive('py33','zip','Python-3.3.0')
'/Users/beazley/Downloads/py33.zip'
>>>
```

`make_archive()` 的第二个参数是期望的输出格式。 可

以使用 `get_archive_formats()` 获取所有支持的归档格式列表。

例如：

```py
>>> shutil.get_archive_formats()
[('bztar', "bzip2'ed tar-file"), ('gztar', "gzip'ed tar-file"),
 ('tar', 'uncompressed tar file'), ('zip', 'ZIP file')]
>>>
```

`Python` 还有其他的模块可用来处理多种归档格式（比如 `tarfile` , `zipfile` , `gzip` , `bz2` ）的底层细节。 

不过，如果你仅仅只是要创建或提取某个归档，就没有必要使用底层库了。 

可以直接使用 `shutil` 中的这些高层函数。


# [os.walk()]通过文件名查找文件

查找文件，可使用 `os.walk()` 函数，传一个顶级目录名给它。 下面是一个例子，查找特定的文件名并答应所有符合条件的文件全路径：

In [40]:
%%file findfile.py

import os
import sys

def findfile(start, name):
    for relpath, dirs, files in os.walk(start):
        if name in files:
            full_path = os.path.join(start, relpath, name)
            print(os.path.normpath(os.path.abspath(full_path)))

if __name__ == '__main__':
    findfile(sys.argv[1], sys.argv[2])
    os.system("pause")

Overwriting findfile.py


In [41]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))

# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'findfile.py')
cmd = "python " + f'"{DIR}"'  \
+ r" D:/CODE/GitHub/py/資料結構/py3-cookbook/脚本编程与系统管理" \
+ " findfile.py" 
# sys.argv[1]  # sys.argv[2]


# 印出 cmd DIR
print(cmd,BASE_DIR,sep='\n')

#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

# 暫停(需在要執行的 py 上)
#subprocess.call("pause",shell=True)
#os.system("pause")

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理\findfile.py" D:/CODE/GitHub/py/資料結構/py3-cookbook/脚本编程与系统管理 findfile.py
D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理


In [34]:
import os

def findfile(start, name):
    for relpath, dirs, files in os.walk(start):
        if name in files:
            full_path = os.path.join(start, relpath, name)
            print(os.path.normpath(os.path.abspath(full_path)))
            
if __name__ == '__main__':
    findfile(
        r"D:/CODE/GitHub/py/資料結構/py3-cookbook/脚本编程与系统管理",
        "findfile.py"
    )

D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理\findfile.py


`os.walk()` 方法为我们遍历目录树， 每次进入一个目录，它会返回一个三元组，包含相对于查找目录的相对路径，一个该目录下的目录名列表， 以及那个目录下面的文件名列表。

对于每个元组，只需检测一下目标文件名是否在文件列表中。

如果是就使用 `os.path.join()` 合并路径。 

为了避免奇怪的路径名比如 `././foo//bar` ，使用了另外两个函数来修正结果。 

第一个是 `os.path.abspath()` ,它接受一个路径，可能是相对路径，最后返回绝对路径。 

第二个是 `os.path.normpath()` ，用来返回正常路径，可以解决双斜杆、对目录的多重引用的问题等。

下面的函数打印所有最近被修改过的文件：

In [43]:
import os
import time

def modified_within(top, seconds):
    now = time.time()
    for path, dirs, files in os.walk(top):
        for name in files:
            fullpath = os.path.join(path, name)
            if os.path.exists(fullpath):
                mtime = os.path.getmtime(fullpath)
                if mtime > (now - seconds):
                    print(fullpath)

if __name__ == '__main__':
    #import sys
    #if len(sys.argv) != 3:
    #    print('Usage: {} dir seconds'.format(sys.argv[0]))
    #    raise SystemExit(1)

    #modified_within(sys.argv[1], float(sys.argv[2]))
    
    modified_within(
        r"D:/CODE/GitHub/py/資料結構/py3-cookbook/脚本编程与系统管理",
        float('20191010')
    )

D:/CODE/GitHub/py/資料結構/py3-cookbook/脚本编程与系统管理\filein.py
D:/CODE/GitHub/py/資料結構/py3-cookbook/脚本编程与系统管理\FileinputTest.txt
D:/CODE/GitHub/py/資料結構/py3-cookbook/脚本编程与系统管理\findfile.py
D:/CODE/GitHub/py/資料結構/py3-cookbook/脚本编程与系统管理\search.py
D:/CODE/GitHub/py/資料結構/py3-cookbook/脚本编程与系统管理\脚本编程与系统管理.ipynb
D:/CODE/GitHub/py/資料結構/py3-cookbook/脚本编程与系统管理\.ipynb_checkpoints\脚本编程与系统管理-checkpoint.ipynb


# [configparser.ConfigParser() ; .ini]读取配置文件

In [44]:
%%file config.ini
; config.ini
; Sample configuration file

[installation]
library=%(prefix)s/lib
include=%(prefix)s/include
bin=%(prefix)s/bin
prefix=/usr/local

# Setting related to debug configuration
[debug]
log_errors=true
show_warnings=False

[server]
port: 8080
nworkers: 32
pid-file=/tmp/spam.pid
root=/www/root
signature:
    =================================
    Brought to you by the Python Cookbook
    =================================

Writing config.ini


In [45]:
from configparser import ConfigParser

In [48]:
cfg = ConfigParser()
cfg.read('config.ini')
# ['config.ini']

cfg.sections()
# ['installation', 'debug', 'server']

cfg.get('installation','library')
# '/usr/local/lib'

cfg.getboolean('debug','log_errors')
# True

cfg.getint('server','port')
# 8080

cfg.getint('server','nworkers')
# 32

print(cfg.get('server','signature'))
# =================================
# Brought to you by the Python Cookbook
# =================================

['config.ini']

['installation', 'debug', 'server']

'/usr/local/lib'

True

8080

32


Brought to you by the Python Cookbook


如果有需要，你还能修改配置并使用 `cfg.write()` 方法将其写回到文件中。

例如：

In [49]:
cfg.set('server','port','9000')
cfg.set('debug','log_errors','False')
import sys
cfg.write(sys.stdout) # sys.stdout.write 方法把字符写入到标准输出中

[installation]
library = %(prefix)s/lib
include = %(prefix)s/include
bin = %(prefix)s/bin
prefix = /usr/local

[debug]
log_errors = False
show_warnings = False

[server]
port = 9000
nworkers = 32
pid-file = /tmp/spam.pid
root = /www/root
signature = 
	Brought to you by the Python Cookbook



In [ ]:
# %load config.ini
; config.ini
; Sample configuration file

[installation]
library=%(prefix)s/lib
include=%(prefix)s/include
bin=%(prefix)s/bin
prefix=/usr/local

# Setting related to debug configuration
[debug]
log_errors=true
show_warnings=False

[server]
port: 8080
nworkers: 32
pid-file=/tmp/spam.pid
root=/www/root
signature:
    =================================
    Brought to you by the Python Cookbook
    =================================

## [ini]讨论

配置数据会被分组（比如例子中的 `installation` 、 `debug` 和 `server`）。 

个分组在其中指定对应的各个变量值。

下面的赋值语句是等效的：

```ini
prefix=/usr/local
prefix: /usr/local
```    
    
配置文件中的名字是不区分大小写的。例如：

```
>>> cfg.get('installation','PREFIX')
'/usr/local'
>>> cfg.get('installation','prefix')
'/usr/local'
```

在解析值的时候， `getboolean()` 方法查找任何可行的值。

例如下面都是等价的：

```ini
log_errors = true
log_errors = TRUE
log_errors = Yes
log_errors = 1
```

它并不是从上而下的顺序执行。 

文件是安装一个整体被读取的。

如果碰到了变量替换，它实际上已经被替换完成了。 

例如，在下面这个配置中， `prefix` 变量在使用它的变量之前或之后定义都是可以的：    

```ini
[installation]
library=%(prefix)s/lib
include=%(prefix)s/include
bin=%(prefix)s/bin
prefix=/usr/local
```

`ConfigParser` 有个容易被忽视的特性是它能一次读取多个配置文件然后合并成一个配置。 

例如，假设一个用户像下面这样构造了他们的配置文件：

```ini
; ~/.config.ini
[installation]
prefix=/Users/beazley/test

[debug]
log_errors=False
```

读取这个文件，它就能跟之前的配置合并起来。如：

```py
>>> # Previously read configuration
>>> cfg.get('installation', 'prefix')
'/usr/local'

>>> # Merge in user-specific configuration
>>> import os
>>> cfg.read(os.path.expanduser('~/.config.ini'))
['/Users/beazley/.config.ini']

>>> cfg.get('installation', 'prefix')
'/Users/beazley/test'
>>> cfg.get('installation', 'library')
'/Users/beazley/test/lib'
>>> cfg.getboolean('debug', 'log_errors')
False
```

仔细观察下 `prefix` 变量是怎样覆盖其他相关变量的，比如 `library` 的设定值。

产生这种结果的原因是变量的改写采取的是后发制人策略，以最后一个为准。 

可以像下面这样做试验：

```py
>>> cfg.get('installation','library')
'/Users/beazley/test/lib'
>>> cfg.set('installation','prefix','/tmp/dir')
>>> cfg.get('installation','library')
'/tmp/dir/lib'
>>>
```

# [logging]给简单脚本增加日志功能

打印日志最简单方式是使用 `logging` 模块

In [52]:
import logging

def main():
    # Configure the logging system
    logging.basicConfig(
        filename='app.log',
        level=logging.ERROR
    )

    # Variables (to make the calls that follow work)
    hostname = 'www.python.org'
    item = 'spam'
    filename = 'data.csv'
    mode = 'r'

    # Example logging calls (insert into your program)
    logging.critical('Host %s unknown', hostname)
    logging.error("Couldn't find %r", item)
    logging.warning('Feature is deprecated')
    logging.info('Opening file %r, mode=%r', filename, mode)
    logging.debug('Got here')

if __name__ == '__main__':
    main()

上面五个日志调用 `（critical(), error(), warning(), info(), debug()）` 以降序方式表示不同的严重级别。

`basicConfig()` 的 `level` 参数是一个过滤器。 

所有级别低于此级别的日志消息都会被忽略掉。 

每个 `logging` 操作的参数是一个消息字符串，后面再跟一个或多个参数。

构造最终的日志消息的时候我们使用了 `%` 操作符来格式化消息字符串。

如果你想改变输出等级，你可以修改 `basicConfig()` 调用中的参数。

例如：

```py
logging.basicConfig(
     filename='app.log',
     level=logging.WARNING,
     format='%(levelname)s:%(asctime)s:%(message)s'
)
```

上面的日志配置都是硬编码到程序中的。

如果你想使用配置文件， 可以像下面这样修改 `basicConfig()` 调用：

```py
import logging
import logging.config

def main():
    # Configure the logging system
    logging.config.fileConfig('logconfig.ini')
    ...
```

创建一个下面这样的文件，名字叫 `logconfig.ini` ：

```ini
[loggers]
keys=root

[handlers]
keys=defaultHandler

[formatters]
keys=defaultFormatter

[logger_root]
level=INFO
handlers=defaultHandler
qualname=root

[handler_defaultHandler]
class=FileHandler
formatter=defaultFormatter
args=('app.log', 'a')

[formatter_defaultFormatter]
format=%(levelname)s:%(name)s:%(message)s
```



尽管对于 `logging` 模块而已有很多更高级的配置选项， 不过这里的方案对于简单的程序和脚本已经足够了。 

只想在调用日志操作前先执行下 `basicConfig()` 函数方法，你的程序就能产生日志输出了。

如果你想要你的日志消息写到标准错误中，而不是日志文件中，调用 `basicConfig()`  时不传文件名参数即可。

例如：

```py
logging.basicConfig(level=logging.INFO)
```

`basicConfig()` 在程序中只能被执行一次。

如果你稍后想改变日志配置， 就需要先获取 `root logger` ，然后直接修改它。

例如：

```py
logging.getLogger().level = logging.DEBUG
```

# [logging]给函数库增加日志功能

对于想要执行日志操作的函数库而已，你应该创建一个专属的 `logger` 对象，并且像下面这样初始化配置：

In [1]:
%%file somelib.py

import logging
log = logging.getLogger(__name__)
log.addHandler(logging.NullHandler())

# Example function (for testing)
def func():
    print('--- in somelib.func() ---')
    log.critical('A Critical Error!')
    log.debug('A debug message')

Overwriting somelib.py


In [2]:
%%file __init__.py

# for import somelib.py

Overwriting __init__.py


使用这个配置，默认情况下不会打印日志。例如：

In [3]:
import somelib
somelib.func()

--- in somelib.func() ---


In [4]:
import logging
logging.basicConfig()
somelib.func()

CRITICAL:somelib:A Critical Error!


--- in somelib.func() ---


## 讨论

通常来讲，你不应该在函数库代码中自己配置日志系统，或者是已经假定有个已经存在的日志配置了。

调用 `getLogger(__name__)` 创建一个和调用模块同名的 `logger` 模块。 

由于模块都是唯一的，因此创建的 `logger` 也将是唯一的。

`log.addHandler(logging.NullHandler())` 操作将一个空处理器绑定到刚刚已经创建好的 `logger` 对象上。

一个空处理器默认会忽略调用所有的日志消息。 

因此，如果使用该函数库的时候还没有配置日志，那么将不会有消息或警告出现。

还有一点就是对于各个函数库的日志配置可以是相互独立的，不影响其他库的日志配置。 例如，对于如下的代码：

```py
>>> import logging
>>> logging.basicConfig(level=logging.ERROR)

>>> import somelib
>>> somelib.func()
CRITICAL:somelib:A Critical Error!

>>> # Change the logging level for 'somelib' only
>>> logging.getLogger('somelib').level=logging.DEBUG
>>> somelib.func()
CRITICAL:somelib:A Critical Error!
DEBUG:somelib:A debug message
>>>
```

在这里，根日志被配置成仅仅输出 `ERROR` 或更高级别的消息。

不过 ， `somelib` 的日志级别被单独配置成可以输出 `debug` 级别的消息，它的优先级比全局配置高。 

像这样更改单独模块的日志配置对于调试来讲是很方便的， 因为你无需去更改任何的全局日志配置——只需要修改你想要更多输出的模块的日志等级。

# [Timer]实现一个计时器

`time` 模块包含很多函数来执行跟时间有关的函数。 

尽管如此，通常我们会在此基础之上构造一个更高级的接口来模拟一个计时器

In [5]:
import time

class Timer:
    def __init__(self, func=time.perf_counter):
        self.elapsed = 0.0
        self._func = func
        self._start = None

    def start(self):
        if self._start is not None:
            raise RuntimeError('Already started')
        self._start = self._func()

    def stop(self):
        if self._start is None:
            raise RuntimeError('Not started')
        end = self._func()
        self.elapsed += end - self._start
        self._start = None

    def reset(self):
        self.elapsed = 0.0

    @property
    def running(self):
        return self._start is not None

    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

这个类定义了一个可以被用户根据需要启动、停止和重置的计时器。 

它会在 `elapsed` 属性中记录整个消耗时间。 

下面是一个例子来演示怎样使用它：

In [6]:
def countdown(n):
    while n > 0:
        n -= 1

# Use 1: Explicit start/stop
t = Timer()
t.start()
countdown(1000000)
t.stop()
print(t.elapsed)

# Use 2: As a context manager
with t:
    countdown(1000000)
print(t.elapsed)

with Timer() as t2:
    countdown(1000000)
print(t2.elapsed)

0.0584096
0.1169806
0.0565378


## 讨论

本节提供了一个简单而实用的类来实现时间记录以及耗时计算。 

同时也是对使用 `with` 语句以及上下文管理器协议的一个很好的演示。

在计时中要考虑一个底层的时间函数问题。

一般来说， 使用 `time.time()` 或 `time.clock()` 计算的时间精度因操作系统的不同会有所不同。 

而使用 `time.perf_counter()` 函数可以确保使用系统上面最精确的计时器。

上述代码中由 `Timer` 类记录的时间是钟表时间，并包含了所有休眠时间。

如果你只想计算该进程所花费的 `CPU` 时间，应该使用 `time.process_time()` 来代替：

```py
t = Timer(time.process_time)
with t:
    countdown(1000000)
print(t.elapsed)
```

`time.perf_counter()` 和 `time.process_time()` 都会返回小数形式的秒数时间。 

实际的时间值没有任何意义，为了得到有意义的结果，你得执行两次函数然后计算它们的差值。

# 限制内存和CPU的使用量

`resource` 模块能同时执行这两个任务。例如，要限制 `CPU` 时间，可以像下面这样做：


In [8]:
# ModuleNotFoundError: No module named 'resource'
!pip install python-resources

[**stackoverflow**](https://stackoverflow.com/questions/37710848/importerror-no-module-named-resource)

`resource` 是一個特定於 `Unix` 的軟件包，如

https://docs.python.org/2/library/resource.html

中所示，這就是為什麼它在 `Ubuntu` 中為您工作的原因，但是在嘗試在 `Windows` 中使用它時出現了錯誤。

In [2]:

import signal
import resource
import os

def time_exceeded(signo, frame):
    print("Time's up!")
    raise SystemExit(1)

def set_max_runtime(seconds):
    # Install the signal handler and set a resource limit
    soft, hard = resource.getrlimit(resource.RLIMIT_CPU)
    resource.setrlimit(resource.RLIMIT_CPU, (seconds, hard))
    signal.signal(signal.SIGXCPU, time_exceeded)

if __name__ == '__main__':
    set_max_runtime(15)
    while True:
        pass

ModuleNotFoundError: No module named 'resource'

# [webbrowser]启动一个WEB浏览器

`webbrowser` 模块能被用来启动一个浏览器，并且与平台无关

In [3]:
import webbrowser
webbrowser.open('http://www.python.org')

True

它会使用默认浏览器打开指定网页。

如果你还想对网页打开方式做更多控制，还可以使用下面这些函数：

In [7]:
# Open the page in a new browser window
webbrowser.open_new('http://www.python.org')

True

In [5]:
# Open the page in a new browser tab
webbrowser.open_new_tab('http://www.python.org')

True

?上述結果都是 打开一个新标签 tab

In [8]:
c = webbrowser.get('firefox')

Error: could not locate runnable browser